# Daten-Pipeline Part 1
- Einlesen Rohdaten<br>
- grobes Filtern<br>
- Bereinigung nach Titel und Autor<br>
- Export in kleinere JSON-Dateien

### Imports und Funktionen:

In [5]:
# nötige imports
import numpy as np
import pandas as pd
import fileinput
import json
from tqdm import tqdm
tqdm.pandas()
from tssplit import tssplit
import re

In [6]:
# Unique list erstellen für cleaning
def remove_duplicates(l):
    return list(set(l))

### Datenimport:

In [7]:
%%time
# Zeit ausgeben lassen (muss alleine in der ersten Zeile stehen)

# Daten einlesen 
# ACHTUNG: Benötigt über 100GB RAM!
df = pd.read_json('/home/dave/NLP/WashingtonPost-v4/data/TREC_Washington_Post_collection.v4.jl', lines=True)

CPU times: user 3min 12s, sys: 2min 44s, total: 5min 56s
Wall time: 5min 56s


### Datenverarbeitung:

In [8]:
# Nicht verwendete Spalten löschen
df.drop('article_url', inplace=True, axis=1)
df.drop('published_date', inplace=True, axis=1)
df.drop('content', inplace=True, axis=1)
df.drop('orig-id', inplace=True, axis=1)
df.drop('id', inplace=True, axis=1)
df.drop('type', inplace=True, axis=1)
df.drop('source', inplace=True, axis=1)


In [11]:
# RegEx Pattern und Variablen erstellen
pattern_auth = "\s*\w*"
pattern_at_empty = "" #(\:*\s*'')
drop_list = []
index = 0
i = 0


# Leere oder nicht gewollte Zeilen mit Hilfe von Autoren- und Titelspalte aussortieren
for auth in df["author"]:
    at = re.search(pattern_auth, str(auth))
    if at or len(str(auth)) <= 4:
        drop_list.append(i)
    if auth == "":
        drop_list.append(i)
    i+= 1
    

for tit in df["title"]:
  i = 0
  if len(str(tit)) <= 4:
    drop_list.append(i)
  if auth == "":
    drop_list.append(i)
  i+= 1

df = df.drop(remove_duplicates(drop_list))  

# Indexreset, da Zeilen gelöscht wurden
df = df.sort_index()
df = df.reset_index(drop=True)

### Datenexport:

In [14]:
%%time

# Datei ist zu groß, um sie auf einem PC zu handhaben -> Auteilen in 8 Dateien
le = int(len(df)/8)
start = 0
end = le
i = 1 
while i < 9:
    df[start:end].to_json("/home/dave/NLP/WashingtonPost-v4/data/"+str(i)+".json")
    start = end
    end = end + le
    i += 1

CPU times: user 41.5 s, sys: 6.49 s, total: 48 s
Wall time: 48 s


# Daten-Pipeline Part 2
- Loop durch o.g. Export<br>
  - Aufteilen der "content"-Spalte in Meta- und Inhaltslisten<br>
    - Aus Content leserlichen Text extrahieren<br>
  - Filtern der erstellten Listen -> Erstellung neues, ordentliches Dataframe <br>
  - Content und Meta mergen
  - Export zu JSON

In [1]:
#Erneute imports wegen Neustart Kernel
import numpy as np
import pandas as pd
import fileinput
import json
from tqdm import tqdm
tqdm.pandas()

from tssplit import tssplit
import re

In [2]:
%%time

# RegEx-Pattern 
pattern_paragraph = "\'subtype\': \'paragraph\'"
pattern_date = "\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2}Z"


# Loop durch in Part 1 erstellte Exports
for c in range(1,9):
    
    # Import in DataFrame
    df = pd.read_json("/home/dave/NLP/WashingtonPost-v4/data/"+str(c)+".json")
    
    # Indexreset 
    df = df.reset_index(drop=True)
    
    # Variablen erstellen
    meta_List = []
    content_List = []
    j= 0

    # Row-Wise Loop durch "content"
    for article in df["contents"]:
        
        # temporäre Variablen erstellen
        temp_List_meta = []
        temp_List_content = []
        temp_List_content2 = []
        temp_List_meta2 = []
        temp_List_meta3 = []

        # Loop durch "article", Inhalt mit RegEx von Meta trennen
        # Meta und Content werden jeweils in temporäre Listen geschrieben
        for eintrag in article:
            x = re.search(pattern_paragraph, str(eintrag))
            if x:
                temp_List_content = [eintrag]
                for t in temp_List_content:
                    val = str(t)
                    val3 = tssplit(val,quote="'", delimiter=",")
                    t4 = val3[0][10:]
                    temp_List_content2.append(t4)
            else:
                temp_List_meta = [eintrag]
                for t in temp_List_meta:

                    val2 = str(t)
                    val6 = val2.split(",")
                    t8 = val6[0][13:]
                    temp_List_meta2.append(t8)
        
        
        
        # Zusammenführung der temporären Metaliste 3 aus temporäre Metaliste 2 und Teilen des ursprünglichen DFs      
        temp_List_meta3.append(temp_List_meta2[0])
        temp_List_meta3.append(df["title"][j])   
        temp_List_meta3.append(df["author"][j])

        for m in temp_List_meta2:                
            d = re.search(pattern_date, str(m))          
            if d:
                temp_List_meta3.append(m)
            else:
                continue
        
        
        # Filterung der erstellten Listen, um leere und nicht gewollte Einträge auszuschließen
        if len(3) >= 3 and len(temp_List_meta3) <= 4 and len(temp_List_content2) > 3:
            meta_List.append(temp_List_meta3)    
            content_List.append("".join(temp_List_content2))
        j+=1        

    # Neue DFs aus den 2 Listen 
    dfMeta = pd.DataFrame(meta_List)
    dfContent = pd.DataFrame(content_List)

    # Bennenung der Spalten für beide DFs
    dfContent = dfContent.set_axis(["content"], axis=1, inplace = False)
    dfMeta = dfMeta.set_axis(["Kategorie","Titel","Autor","Datum"], axis=1, inplace = False)

    # Temporäre IDs fürs Zusammenführen der eben erstellten DFs erstellen
    dfContent["id"] = dfContent.index
    dfMeta["id1"] = dfMeta.index

    # DFs zusammenführen
    df = pd.merge(dfMeta,dfContent, left_on="id1", right_on="id", how="left").drop("id1",axis=1)
    
    
    # Export des DFs zu einer JSON
    df.to_json("/home/dave/NLP/WashingtonPost-v4/data/v6/"+str(c)+"_raw.json")

    c += 1

CPU times: user 18min 5s, sys: 25.6 s, total: 18min 30s
Wall time: 18min 31s


# Kernel neu starten

# Daten-Pipeline Part 3
- Loop durch o.g. Export<br>
  - Säubern der Daten<br>
  - Export zu JSON 


In [2]:
from tqdm import tqdm
import numpy as np
import pandas as pd
tqdm.pandas()
import fileinput
import re


In [ ]:
!python -m spacy download en_core_web_md 
!python -m spacy download en_core_web_lg

In [5]:
%%time
# Loop durch in Part 2 erstellte Exports
for c in range(1,9):
    
    # Import in DataFrame
    df = pd.read_json("/home/dave/NLP/WashingtonPost-v4/data/v6/"+str(c)+"_raw.json")
    
    # Indexreset  
    df = df.set_index('id')
    
    # Säubern der Daten
    df.Kategorie = df.Kategorie.replace("\"$|\'$", "",regex=True)
    df.content = df.content.replace("\<.*?\>","",regex=True)
    df.Titel = df.Titel.replace("\<.*?\>","",regex=True)
    df.Titel = df.Titel.replace("\"$|\'$", "",regex=True)
    df.Datum = df.Datum.replace("\"$|\'$", "",regex=True) 
    
    # Export zu JSON
    df.to_json("data/v6/clean/"+str(c)+"_clean.json")

CPU times: user 56.7 s, sys: 15.5 s, total: 1min 12s
Wall time: 1min 12s


# Kernel neu starten

# Daten-Pipeline Part 3.2
- Import von Kategorien(Export aus dem Datensatz, intellektuell erschlossen)<br>
- Loop durch o.g. Export<br>
  - Filterung der Daten nach Kategorien<br>
  - Export zu JSON


In [1]:
from tqdm import tqdm
import numpy as np
import pandas as pd
tqdm.pandas()
import fileinput
import re

In [2]:
# Import der Kategorien DF
dfKat_behalten = pd.read_csv("/home/dave/NLP/WashingtonPost-v4/data/Kategorie_behalten.CSV",header=None)

# DF zu Liste
Kat_behalten = list(dfKat_behalten[0])

In [4]:
%%time

# Variable erstellen
temp_list = []

# Loop durch in Part 3 erstellte Exports
for c in range(1,9):
    
    # Import in DataFrame
    df = pd.read_json("/home/dave/NLP/WashingtonPost-v4/data/v6/clean/"+str(c)+"_clean.json")
    
    # Variable erstellen, jeden Loop resetten
    i=0
    dfEnd = pd.DataFrame()
    
    # Filtern nach Kategorien
    for a in df.iterrows():
        if a[1][0] in Kat_behalten:
            temp_list.append(a[1])
        i+= 1

# Temporäre Liste zu DF    
dfEnd = pd.DataFrame(temp_list)

# (Umständlicher) IndexReset
dfEnd = dfEnd.reset_index()
dfEnd = dfEnd.drop("index", axis=1)
dfEnd = dfEnd.reset_index()
dfEnd = dfEnd.set_index('index')  

CPU times: user 52.4 s, sys: 10.2 s, total: 1min 2s
Wall time: 1min 2s


In [7]:
%%time

# Anzahl der zu erstellenden Exports
anzahl = 16

# Variablen
le = int(len(dfEnd)/anzahl)
start = 0
end = le
i = 1 

# Export zu JSON
while i < (anzahl+1):
    dfEnd[start:end].to_json("/home/dave/NLP/WashingtonPost-v4/data/v6/rdy/"+str(anzahl)+"/"+str(i)+"_rdy.json")
    start = end
    end = end + le
    i += 1

CPU times: user 3 s, sys: 2.97 s, total: 5.97 s
Wall time: 5.97 s


In [8]:
# Export zu einer großen JSON
dfEnd.to_json("data/v6/rdy/all_rdy.json")

# Kernel neu starten

In [9]:
dfEnd

,Kategorie,Titel,Autor,Datum,content
index,,,,,
0,Local,Bubbly planet Venus starts off New Year,Blaine P. Friedlander Jr.,2012-01-01T00:12:09Z,"Set down your champagne and gaze west, as the ..."
1,Politics,Iowa caucuses: GOP presidential hopefuls makin...,Karen Tumulty,2012-01-01T01:55:00Z,DES MOINES — Two days before the voting begins...
2,Local,Developer of Maryland housing office in Prince...,Ovetta Wiggins,2012-01-01T00:56:45Z,A developer who stands to gain millions by bui...
3,Politics,Many Iowans still don’t know who they will cau...,Jason Horowitz,2012-01-01T01:17:00Z,"Early last week, a postcard advertising a rall..."
4,Local,"As homicides fall in D.C., rise in Prince Geor...",Allison Klein; Matt Zapotosky,2012-01-01T01:37:52Z,The District and Prince George’s County had ne...
...,...,...,...,...,...
229695,The Americas,"She’s young, has no serious health conditions ...",Terrence McCoy; Heloísa Traiano,2020-12-31T11:04:10Z,"On the 265th day of her isolation, Barbie Furt..."
229696,Politics,Sen. David Perdue became wealthy outsourcing w...,Michael Kranish,2020-12-31T12:22:59Z,When Republican David Perdue ran for the Senat...
229697,Local,Leaders in Washington region pledge to increas...,Meagan Flynn; Rachel Chason; Rebecca Tan,2020-12-31T10:38:09Z,Vaccine rollout has moved slowly in the Washin...
